In [1]:
import csv
from bs4 import BeautifulSoup
import requests
import ollama
import openai
import psycopg2

from pgvector.psycopg2 import register_vector
from dotenv import load_dotenv
import os
import numpy as np
from rich.jupyter import print
from psycopg2.extras import RealDictCursor

## First steps

In [2]:
output = ollama.generate(
  model="llama3.1",
  prompt=f"What is the second family name of Javier Martinez according to spanish rule of family names? you are force to provide an answer and provide a single word answer"
)
output.get("response")

'Martinez.\n\n(Note: In Spanish, the rule is to list surnames in order of frequency or importance. If both surnames have equal importance, they can be listed in alphabetical order. However, if one surname is more common than the other, it is typically listed first.)\n\nBut, considering that "Martinez" is a very common surname, I\'m going to take an educated guess and say that... (drum roll)... Javier Martinez is probably from Spain or a Spanish-speaking country where the rule of listing surnames in order of importance might apply.\n\nSo, my answer is: Martínez.'

In [6]:
data = "Virginia Alcantara Vila is the mother of Javier Martinez. Both are spaniards"

prompt= "What is the mother's family name of Javier Martinez (a fiction character) according to spanish rule of family names? you are force to provide an answer and provide a single word answer"

output = ollama.generate(
  model="llama3.1",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}",
)
output.get("response")

'Vila'

# Data Extraction

In [ ]:

url="https://tow.whfb.app"

html_content = requests.get(url).text


soup = BeautifulSoup(html_content)

soup.findAll("a")
links = soup.findAll("a")

with open(f"old_world_rules3.csv", 'w', newline='') as out_file:
    headers = [
        "rule_name", 
        "link_number",
        "link",
        "rulebook_page",
        "rule_description"
    ]
    writer = csv.DictWriter(out_file, headers)
    writer.writeheader()
    total_links = len(links)
    
    id = 0
    for number, link in enumerate(links):
        try:
            # if number == 20: 
            #     break
            l = url + link.get("href")

            print(f"Processing {number} out of {total_links}")
            html_content = requests.get(l).text


            # Parse the html content
            subsoup = BeautifulSoup(html_content)
            rule_name = subsoup.find("h1", attrs={"class": "page-title"}).text.removesuffix("URL Copied!")
            rulebook_page = subsoup.find("li", attrs={"class": "page-reference"}).text
            rule_description = ""

            if subsoup.find("article", attrs={"class": "article--rich-text"}):
                rule_description = subsoup.find("article", attrs={"class": "article--rich-text"}).text
            
            if subsoup.find("article", attrs={"class": "article--rich-text section-intro"}):
                rule_description += subsoup.find("article", attrs={"class": "article--rich-text section-intro"}).text
            
            row = {
                "rule_name": rule_name,
                "link_number": str(id),
                "link": l,
                "rulebook_page": rulebook_page,
                "rule_description": rule_description
                }

            writer.writerow(row)
            id += 1

            spells = subsoup.find("div", attrs={"class": "spell-list"}).findAll("a")
            spells = [ spell for spell in spells if "/spell" in spell.get("href")]
            # print(spells)
            if spells:
                for spell in spells:
                    print(spell)
                    spell_link = url + spell.get("href")

                    html_content = requests.get(spell_link).text 
                    spell = BeautifulSoup(html_content)
                    
                    spell_name = spell.find("h1", attrs={"class": "page-title"}).text.removesuffix("URL Copied!")
                    spell_rulebook_page = spell.find("li", attrs={"class": "page-reference"}).text

                    descriptions = spell.findAll("article", attrs={"class": "article--rich-text"})
                    spell_description = descriptions[0].text
                    
                    spell_info = [el.text for el in descriptions[1].findAll("p") if el.text]
                    # print(spell_info)
                    spell_description += " " + spell_info.pop()
                    # print(spell_info)
                    for info in range(0,len(spell_info),2):
                        spell_description += f" {spell_info[info]}:{spell_info[info+1].replace('"', " inches")},"
                    
                    spell_description = spell_description.removesuffix(",")
                    row = {
                        "rule_name": spell_name,
                        "link_number": str(id),
                        "link": spell_link,
                        "rulebook_page": spell_rulebook_page,
                        "rule_description": spell_description
                        }

                    writer.writerow(row)
                    id += 1

        except Exception as e:
            print(e)
            continue

# Database initialization

In [2]:
# Set up Postgres
load_dotenv(override=True)
DBUSER = os.environ ["DBUSER"]
DBPASS = os.environ["DBPASS"]
DBHOST = os.environ["DBHOST"]
DBNAME = os.environ ["DBNAME" ]
# Use SSL if not connecting to localhost
DBSSL = "disable"
if DBHOST != "localhost":
    DBSSL = "require"
    
conn = psycopg2. connect (database=DBNAME, user=DBUSER, password=DBPASS, host=DBHOST, sslmode=DBSSL)
conn.autocommit = True
cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


# Embeddings generation

In [5]:
# Set up Github models
endpoint = "https://models.inference.ai.azure.com"
model_name = "text-embedding-3-small"

Generating the table that will contain the embeddings

In [10]:
EMBEDDINGS_SIZE = 1024

cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
cur.execute("DROP TABLE IF EXISTS rules")
cur.execute(f"""CREATE TABLE rules (
            link_number int PRIMARY KEY,
            rule_name varchar,
            link varchar,
            rulebook_page varchar,
            rule_description varchar,
            embedding vector({EMBEDDINGS_SIZE})
            );""")
register_vector(conn)

cur.execute("CREATE INDEX ON rules USING hnsw (embedding vector_cosine_ops)")



Reading rules from our disk and load them into memory:

In [11]:
with open(f"old_world_rules3.csv", 'r', newline='') as csvfile:
    rules = csv.DictReader(csvfile, delimiter=',')


    # Iterating the rules
    for rule in rules:
        link_number = rule.get("link_number")
        rule_name = rule["rule_name"]
        link = rule["link"]
        rulebook_page = rule["rulebook_page"]
        rule_description = rule["rule_description"]

        embedding = ollama.embeddings(
            model="mxbai-embed-large", 
            prompt=f"Rule: {rule_name}. Rule Description: {rule_description}"
            )
        # print(f"{link_number}, {rule_name}, {link}, {rulebook_page}, {rule_description}, {embedding.get("embedding")}")
        cur.execute(
            "INSERT INTO rules (link_number, rule_name, link, rulebook_page, rule_description, embedding) VALUES (%s,%s,%s,%s,%s,%s)", 
            (link_number, rule_name, link, rulebook_page, rule_description, embedding.get("embedding"),)
            )
        


# Comparison with embeddings using vector search

In [73]:
query = "SELECT rule_name, embedding <-> %s as distance FROM rules ORDER BY distance LIMIT 20"

prompt = "how to charge a unit?"
cur = conn.cursor(cursor_factory=RealDictCursor)
embedding = np.array(ollama.embeddings(model="mxbai-embed-large", prompt=prompt).get("embedding"))
cur.execute(query, (embedding,))
results=cur.fetchall()
print(results)


[
    RealDictRow({'rule_name': 'The Charge Move', 'distance': 13.069721762651453}),
    RealDictRow({'rule_name': 'The Charge Move (Intro)', 'distance': 13.238372307152169}),
    RealDictRow({'rule_name': 'Declare Charges & Charge Reactions', 'distance': 13.890500869453804}),
    RealDictRow({'rule_name': 'Charging More than One Unit', 'distance': 13.989492560475858}),
    RealDictRow({'rule_name': 'Accidental Contact During a Charge', 'distance': 14.089885985520285}),
    RealDictRow({'rule_name': 'Charging Through Terrain', 'distance': 14.110138493323914}),
    RealDictRow({'rule_name': 'Manoeuvring During a Charge', 'distance': 14.145107844330292}),
    RealDictRow({'rule_name': 'Unable to Align', 'distance': 14.164234326557056}),
    RealDictRow({'rule_name': 'Charge Reactions & Multiple Charging Units', 'distance': 14.221697088984152}),
    RealDictRow({'rule_name': 'Multiple Charge Targets', 'distance': 14.425002200227302}),
    RealDictRow({'rule_name': 'Multiple Charging Units', 'distance': 14.453441719502747}),
    RealDictRow({'rule_name': 'Aligning to the Enemy', 'distance': 14.47902820255661}),
    RealDictRow({'rule_name': 'Running Down the Foe', 'distance': 14.548411614935086}),
    RealDictRow({'rule_name': 'Halting a Charge', 'distance': 14.60165707316216}),
    RealDictRow({'rule_name': 'Charging Units', 'distance': 14.731387016426334}),
    RealDictRow({'rule_name': 'Unusual Situations (Charging)', 'distance': 14.738603747631167}),
    RealDictRow({'rule_name': 'Resolving Uncertainties', 'distance': 14.759284766424363}),
    RealDictRow({'rule_name': 'Disordered Charges', 'distance': 14.833527524441358}),
    RealDictRow({'rule_name': 'Who Can Charge?', 'distance': 14.845497430039604}),
    RealDictRow({'rule_name': 'Flank & Rear Charges', 'distance': 14.8855414984852})
]

In [118]:
query = "SELECT rule_name, embedding <-> %s as distance FROM rules ORDER BY distance LIMIT 20"

prompt = "what is an asrai?"
cur = conn.cursor(cursor_factory=RealDictCursor)
embedding = np.array(ollama.embeddings(model="mxbai-embed-large", prompt=prompt).get("embedding"))
cur.execute(query, (embedding,))
results=cur.fetchall()
print(results)

[
    RealDictRow({'rule_name': 'Corporeal Unmaking', 'distance': 17.289837378139218}),
    RealDictRow({'rule_name': 'Barding', 'distance': 17.346115583325222}),
    RealDictRow({'rule_name': 'Morning Star', 'distance': 17.366536446130116}),
    RealDictRow({'rule_name': 'Unquiet Spirits', 'distance': 17.40040542305606}),
    RealDictRow({'rule_name': 'Resolving Uncertainties', 'distance': 17.5100260760398}),
    RealDictRow({'rule_name': 'Lone Models', 'distance': 17.645923937039875}),
    RealDictRow({'rule_name': 'Suspicious Allied Contingents', 'distance': 17.646030297319175}),
    RealDictRow({'rule_name': 'Fiery Convocation', 'distance': 17.65670197845168}),
    RealDictRow({'rule_name': 'Initiative (I)', 'distance': 17.669795688900876}),
    RealDictRow({'rule_name': 'The Games Master', 'distance': 17.67895560239366}),
    RealDictRow({'rule_name': 'Spirit Leech', 'distance': 17.679840262541525}),
    RealDictRow({'rule_name': 'Redress the Ranks', 'distance': 17.691159142814985}),
    RealDictRow({'rule_name': 'Travel Mystical Pathway', 'distance': 17.73839178767034}),
    RealDictRow({'rule_name': 'Howdah', 'distance': 17.759774109374693}),
    RealDictRow({'rule_name': 'The Role of a GM', 'distance': 17.777795653864032}),
    RealDictRow({'rule_name': 'Mind Razor', 'distance': 17.782721647964475}),
    RealDictRow({'rule_name': 'Talismans', 'distance': 17.79773750927962}),
    RealDictRow({'rule_name': 'Halberd', 'distance': 17.823010365432886}),
    RealDictRow({'rule_name': 'Occupancy', 'distance': 17.829572239407046}),
    RealDictRow({'rule_name': 'Summon Elemental Spirit', 'distance': 17.84733093993854})
]

# Comparison with embeddings using full text search

In [122]:
query = """
    SELECT rule_name, ts_rank_cd(to_tsvector('english', rule_description), query) as rank FROM rules, plainto_tsquery('english', %(prompt)s) query
    WHERE to_tsvector('english', rule_description) @@ query
    ORDER BY rank
    DESC LIMIT 20
    """

prompt = "what is an asrai?"
cur = conn.cursor(cursor_factory=RealDictCursor)
embedding = np.array(ollama.embeddings(model="mxbai-embed-large", prompt=prompt).get("embedding"))
cur.execute(query, {"prompt": prompt})
results=cur.fetchall()
print(results)


[]

In [123]:
query = """
    SELECT rule_name, ts_rank_cd(to_tsvector('english', rule_description), query) as rank FROM rules, plainto_tsquery('english', %(prompt)s) query
    WHERE to_tsvector('english', rule_description) @@ query
    ORDER BY rank
    DESC LIMIT 20
    """

prompt = "lance"
cur = conn.cursor(cursor_factory=RealDictCursor)
embedding = np.array(ollama.embeddings(model="mxbai-embed-large", prompt=prompt).get("embedding"))
cur.execute(query, {"prompt": prompt})
results=cur.fetchall()
print(results)

[
    RealDictRow({'rule_name': 'Lance', 'rank': 0.6}),
    RealDictRow({'rule_name': 'Cavalry Spear', 'rank': 0.2}),
    RealDictRow({'rule_name': 'More than One Combat Weapon', 'rank': 0.1})
]

In [138]:
prompt = "asrai"
cur = conn.cursor(cursor_factory=RealDictCursor)
embedding = np.array(ollama.embeddings(model="mxbai-embed-large", prompt=prompt).get("embedding"))

query = """
WITH semantic_search AS (
    SELECT link_number, rule_name, RANK () OVER (ORDER BY embedding <=> %(embedding)s) AS rank
    FROM rules
    ORDER BY embedding <> %(embedding)s
    LIMIT 20
),
keyword_search AS (
    SELECT link_number, rule_name, RANK () OVER (ORDER BY ts_rank_cd(to_tsvector('english', rule_description), query) DESC) 
    FROM rules, plainto_tsquery('english', %(prompt)s) query 
    WHERE to_tsvector('english', rule_description) @@ query
    ORDER BY ts_rank_cd(to_tsvector('english', rule_description), query) 
    DESC 
    LIMIT 20
)

SELECT
    COALESCE (semantic_search.rule_name, keyword_search.rule_name) AS rule, 
    COALESCE (1.0 / (%(k)s + semantic_search.rank), 0.0) + COALESCE(1.0 / (%(k)s + keyword_search.rank), 0.0) AS rank
FROM semantic_search
FULL OUTER JOIN keyword_search ON semantic_search.link_number = keyword_search.link_number
ORDER BY rank 
DESC
LIMIT 20
"""


cur.execute(
    query, 
    {"prompt": prompt, "embedding": embedding, "k": 60}
    )
results=cur.fetchall()
print(results)

[
    RealDictRow({'rule': 'Killing Blow', 'rank': Decimal('0.01639344262295081967')}),
    RealDictRow({'rule': 'Monster Slayer', 'rank': Decimal('0.01612903225806451613')}),
    RealDictRow({'rule': 'Unquiet Spirits', 'rank': Decimal('0.01587301587301587302')}),
    RealDictRow({'rule': 'Walk Between Worlds', 'rank': Decimal('0.01562500000000000000')}),
    RealDictRow({'rule': 'Initiative (I)', 'rank': Decimal('0.01538461538461538462')}),
    RealDictRow({'rule': 'Vanguard', 'rank': Decimal('0.01515151515151515152')}),
    RealDictRow({'rule': 'Wind Blast', 'rank': Decimal('0.01492537313432835821')}),
    RealDictRow({'rule': 'Attacks (A)', 'rank': Decimal('0.01470588235294117647')}),
    RealDictRow({'rule': 'Combat Result Table', 'rank': Decimal('0.01449275362318840580')}),
    RealDictRow({'rule': 'Unbreakable', 'rank': Decimal('0.01428571428571428571')}),
    RealDictRow({'rule': 'Fiery Convocation', 'rank': Decimal('0.01408450704225352113')}),
    RealDictRow({'rule': 'Inspiring Presence', 'rank': Decimal('0.01388888888888888889')}),
    RealDictRow({'rule': 'Corporeal Unmaking', 'rank': Decimal('0.01369863013698630137')}),
    RealDictRow({'rule': 'Loner', 'rank': Decimal('0.01351351351351351351')}),
    RealDictRow({'rule': 'Highly Manoeuvrable', 'rank': Decimal('0.01333333333333333333')}),
    RealDictRow({'rule': 'Strike Last', 'rank': Decimal('0.01315789473684210526')}),
    RealDictRow({'rule': 'Lone Models', 'rank': Decimal('0.01298701298701298701')}),
    RealDictRow({'rule': 'Impetuous', 'rank': Decimal('0.01282051282051282051')}),
    RealDictRow({'rule': 'Spirit Leech', 'rank': Decimal('0.01265822784810126582')}),
    RealDictRow({'rule': 'Barding', 'rank': Decimal('0.01250000000000000000')})
]

In [12]:
def ragtime(prompt: str) -> list[dict]:
    cur = conn.cursor(cursor_factory=RealDictCursor)
    embedding = np.array(ollama.embeddings(model="mxbai-embed-large", prompt=prompt).get("embedding"))

    query = """
    WITH semantic_search AS (
        SELECT link_number, RANK () OVER (ORDER BY embedding <=> %(embedding)s) AS rank
        FROM rules
        ORDER BY embedding <> %(embedding)s
        LIMIT 20
    ),
    keyword_search AS (
        SELECT link_number, RANK () OVER (ORDER BY ts_rank_cd(to_tsvector('english', rule_description), query) DESC) 
        FROM rules, plainto_tsquery('english', %(prompt)s) query 
        WHERE to_tsvector('english', rule_description) @@ query
        ORDER BY ts_rank_cd(to_tsvector('english', rule_description), query) 
        DESC 
        LIMIT 20
    )

    SELECT
        COALESCE (semantic_search.link_number, keyword_search.link_number) AS link_number, 
        COALESCE (1.0 / (%(k)s + semantic_search.rank), 0.0) + COALESCE(1.0 / (%(k)s + keyword_search.rank), 0.0) AS rank
    FROM semantic_search
    FULL OUTER JOIN keyword_search ON semantic_search.link_number = keyword_search.link_number
    ORDER BY rank 
    DESC
    LIMIT 5
    """


    cur.execute(
        query, 
        {"prompt": prompt, "embedding": embedding, "k": 60}
        )
    results=cur.fetchall()


    rules = [result.get("link_number") for result in results]
    # print(rules)
    cur.execute("SELECT rule_name, link, rulebook_page, rule_description FROM rules WHERE link_number = ANY(%s)", (rules,))
    results= cur.fetchall()

    return [dict(result) for result in results]
    

# Giving shape to the RAG

In [47]:
prompt = "can you explain me the veteran rule and provide me an example"

In [48]:
results= ragtime(prompt)
formatted_results = [f"##{result.get("rule_name")}\n\n{result.get("rule_description")}\n\nRulebook page:{result.get("rulebook_page")}\n\n Reference link:[{result.get("rule_name")}]({result.get("link")})" for result in results]

In [49]:
print(formatted_results)

[
    '##Excess Casualties\n\nIt can sometimes happen that a unit suffers more casualties than it has models in its 
fighting rank. Should this happen, casualties are removed as normal, representing casualties caused in the rank (or
file) behind the fighting rank.In such cases, and if the unit suffering the casualties is able to make supporting 
attacks, any excess casualties will reduce the number of supporting attacks.\n\nRulebook page:Rulebook, p. 150\n\n 
Reference link:[Excess Casualties](https://tow.whfb.app/the-combat-phase/excess-casualties)',
    '##Levies\n\nMany regiments are made up of unwilling fighters, pressed into service.Many regiments are made up 
of unwilling fighters, pressed into service.\n\nRulebook page:Rulebook, p. 172\n\n Reference 
link:[Levies](https://tow.whfb.app/special-rules/levies)',
    '##Veteran\n\nVeteran warriors have seen and done it all, and it takes a lot to unsettle them.Veteran warriors 
have seen and done it all, and it takes a lot to unsettle them.\n\nRulebook page:Rulebook, p. 180\n\n Reference 
link:[Veteran](https://tow.whfb.app/special-rules/veteran)',
    '##Undisciplined\n\nA unit with this rule cannot use the Inspiring Presence rule of the army\'s General nor the
"Hold your Ground" rule of a Battle Standard.\n\nRulebook page:Rulebook, p. 191 & 193\n\n Reference 
link:[Undisciplined](https://tow.whfb.app/troop-types-in-detail/undisciplined)',
    "##Too Many Characters\n\nUnless a unit contains only a single rank, its fighting rank cannot contain more 
characters than it does rank and file models. If it does, one or more must 'retire' from combat. Upon retiring, a 
character:Moves through the ranks to take up a position away from the fighting rank.Cannot make any attacks or have
attacks directed against them.Confers no benefits to the unit in the form of Leadership, special rules, or anything
else.\n\nRulebook page:Rulebook, p. 209\n\n Reference link:[Too Many 
Characters](https://tow.whfb.app/characters/too-many-characters)"
]

In [50]:
system_message = (
    "You must answer questions only according to sources."
    "Say you dont know if you cant find answer in sources."
    "Provide an answer within the scope of the rules of the game called the old world."
    "Cite the source inside square brackets."
    "The sources should contain the reference link and the rulebook page."
    "Place the sources at the end as list of markdowns."
    "The title of each rule will be a markdown heading."
)

In [51]:

client = openai.OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="llama3",
  messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": f"{prompt}\n\n Sources:\n\n {formatted_results}"},
  ],
  temperature=0.3,
  top_p=1.0
)


In [52]:
print(response.choices[0].message.content)

**Veteran Rule**

According to the Warhammer Old World rulebook [1], Veteran warriors have seen and done it all, and it takes a lot 
to unsettle them. This means that Veteran units are more resilient in combat and can withstand more casualties 
before being broken.

Example: A unit of Veteran Warriors is charged by an enemy unit. After the combat phase, they suffer 5 casualties 
but still retain 7 models on the battlefield. As they are Veterans, they are not affected by the excess casualties 
rule [2], and their morale remains unaffected.

Sources:
(https://tow.whfb.app/special-rules/veteran)

In [34]:
formatted_results

["##Forming Units\n\nThe models that make up your army must be formed into 'units' before battle commences, ideally when writing your muster list. A unit usually consists of several models of the same type that have banded together and adopted a specific formation.Additionally, single, powerful models such as a character, a chariot or a Dragon, a war machine and its crew, and so on, are also considered to be a 'unit'. Therefore, whenever the rules that follow refer to 'units', this also includes units of one model.\n\nRulebook page:Rulebook, p. 100\n\n Reference link:[Forming Units](https://tow.whfb.app/forming-units)",
 '##Regular Infantry\n\nThis sub-category covers all the roughly human-sized warriors of the Warhammer world, be they Men, Elves, Dwarfs, Orcs, Zombies and so on. Regular infantry will normally form the core of your Warhammer army, the troops upon which you rely to get the job done. Regular infantry can be expected to adopt almost any formation to fulfil any role.Due to